Ahora que sabes cómo lidiar con los valores ausentes, es momento de aprender algunas técnicas para lidiar con otro problema común en la etapa de preprocesamiento de datos: los valores duplicados.

En el sprint de Python Básico, conociste los métodos duplicated() y drop_duplicates(). En esta lección, revisaremos estos conceptos y otras formas de abordar la duplicación de datos.

Buscar duplicados a mano
El análisis que realizaste previamente sobre las fuentes de tráfico para el sitio de comercio electrónico online generó más preguntas en el equipo a cargo de atraer nuevos clientes. Quieren saber si hay una marca popular de teléfonos móviles que no esté suficientemente representada en el sitio web, así como estadísticas que comprueben qué modelos de teléfonos les interesan más a la clientela.

Para empezar, quieren que averigües cuántas tiendas ya están vendiendo ciertos teléfonos en un mercado online.

Para completar este análisis, utilizamos nuestro viejo amigo read_csv() para leer el conjunto de datos /datasets/phone_stock.csv, guardándolo en la variable df_stock. Imprime las cinco primeras filas de la tabla.

import pandas as pd
df_stock = pd.read_csv('/datasets/phone_stock.csv')
print(df_stock.head(5))
          id                     item  count
0  100480924     Apple iPhone Xr 64gb     10
1  100480924     Apple iPhone Xr 64GB     19
2  100480959     Xiaomi Redmi 6A 16GB     44
3  100480975          HUAWEI P30 lite     38
4  100480988  Samsung Galaxy A30 32GB     49
La tabla tiene:

una columna 'id' con los ID de los productos;
una columna 'item' con los nombres de los modelos;
una columna 'count' con el número total de teléfonos disponibles para determinados modelos.
En las dos primeras filas ya podemos ver datos duplicados.

En el caso del iPhone Xr 64 GB de Apple, solo debería haber una fila de datos y el recuento en esa fila debería ser 29. ¿Qué pasa si los datos de algunos de los otros teléfonos también están duplicados en otra parte de la tabla? Necesitamos poder analizar todo el conjunto de datos a la vez, en lugar de esperar encontrar duplicados leyendo una copia impresa de los datos.

Revisión: encontrar datos duplicados
Hay dos técnicas que funcionan para encontrar datos duplicados:

Técnica 1
Podemos utilizar el método duplicated() junto con sum() para obtener el número de valores duplicados en una sola columna o filas duplicadas en un DataFrame. Recuerda que si llamas a duplicated() sin utilizar sum(), se imprimirá una serie booleana tan larga como el DataFrame, con True donde hay un duplicado y False donde no lo hay. Ejecuta el siguiente código para ver un ejemplo:

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

print('Así es como se ve el dataset:')
print(df)
print('Así es como se ve una serie booleana devuelta:')
print(df.duplicated())
print('Así se ve el resultado de duplicated() con sum():')
print(df.duplicated().sum())
Así es como se ve el dataset:
    col_1  col_2
0     A      1
1     B      2
2     A      2
3     A      1

Así es como se ve una serie booleana devuelta:
0    False
1    False
2    False
3     True
dtype: bool

Así se ve el resultado de duplicated() con sum():
1
Por lo tanto, para comprobar la presencia de duplicados, utiliza el método duplicated() seguido del método sum(). Por cierto, puedes visualizar los duplicados mediante un simple filtrado:

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

print(df[df.duplicated()])
    col_1  col_2
3     A      1
Obtenemos como salida la cuarta fila (la fila con índice 3) porque esta fila es un duplicado de la primera fila (la fila con índice 0).

Técnica 2
Llama al método value_counts(). Este método identifica todos los valores unívocos en una columna y calcula cuántas veces aparece cada uno. Podemos aplicar este método a los Series para obtener los pares valor-frecuencia en orden descendente. Las entradas que se duplican con más frecuencia se encuentran en la parte superior de la lista. A continuación te mostramos un ejemplo:

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

print('Así es como se ve el dataset:')
print(df)
print()
print('Este es el resultado de la llamada al método value_counts() para col_1:')
print(df['col_1'].value_counts())
Así es como se ve el dataset:
  col_1  col_2
0     A      1
1     B      2
2     A      2
3     A      1

Este es el resultado de la llamada al método value_counts() para col_1:
A    3
B    1
Name: col_1, dtype: int64
La letra A aparece tres veces, mientras que la B solo aparece una vez. Sin exploración adicional, no podemos concluir que hay tres filas duplicadas en las que 'col_1' es A. Sin embargo, ahora tenemos una hipótesis de que podría ser el caso. Por ejemplo, en cuanto a teléfonos, hay dos filas con Apple iPhone Xr 64gb que definitivamente vamos a fusionar para tener una idea clara sobre el stock.

Gestión de duplicados
Como ya aprendimos en el sprint de Python Básico, las filas completamente duplicadas, como la primera y la última fila del DataFrame del ejemplo anterior, se pueden tratar utilizando el método drop_duplicates():

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

print('Dataset original:')
print(df)
print()
print('Dataset con duplicados eliminados:')
print(df.drop_duplicates())
Dataset original:
  col_1  col_2
0     A      1
1     B      2
2     A      2
3     A      1

Dataset con duplicados eliminados:
  col_1  col_2
0     A      1
1     B      2
2     A      2
Si solo deseas considerar duplicados en una (o algunas) de las columnas en lugar de filas completamente duplicadas, puedes usar el parámetro subset=. Pásale el nombre de la columna (o la lista de nombres de columna) donde deseas buscar duplicados:

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

print(df)
print()
print(df.drop_duplicates(subset='col_1'))
  col_1  col_2
0     A      1
1     B      2
2     A      2
3     A      1

  col_1  col_2
0     A      1
1     B      2
Pero la naturaleza de los datos duplicados de nuestro teléfono es un poco diferente a este ejemplo de DataFrame. Echémosle otro vistazo:

import pandas as pd

stock = pd.read_csv('/datasets/phone_stock.csv')
print(stock['item'].value_counts())
Apple iPhone Xr 64gb       1
Apple iPhone Xr 64GB       1
Xiaomi Redmi 6A 16GB       1
HUAWEI P30 lite            1
Samsung Galaxy A30 32GB    1
Samsung Galaxy A30 32gb    1
Honor 8X 64GB              1
Name: item, dtype: int64
Hay dos entradas para el Samsung Galaxy y el iPhone de Apple. La única diferencia entre las dos entradas para ambos teléfonos es gb frente a GB. En realidad son duplicados, pero Python no los reconoce como tales porque las cadenas no son idénticas.

La forma más sencilla de manejar entradas duplicadas como estas es hacer que todas las letras de las cadenas estén en minúsculas, utilizando el método lower(). A continuación te mostramos un ejemplo:

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

print('Col_1 original en el dataset:')
print(df['col_1'])
print()
print('Dataset con valores reducidos en col_1')
print(df['col_1'].str.lower())
Observa que después de df['col_1'], tenemos .str, que nos permite aplicar métodos de cadena directamente a la columna. Esto es exactamente lo que hay que hacer para poder aplicar el método lower() en el siguiente paso.

Ten en cuenta que si quieres sustituir la columna original por una nueva en la que todas las letras estén en minúsculas, tendrás que volver a hacer la asignación.

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

# sustituir la columna col_1
df['col_1'] = df['col_1'].str.lower()
print(df)
    col_1  col_2
0     a      1
1     b      2
2     a      2
3     a      1
Debes asegurarte de que las mayúsculas no sean importantes antes de hacer que todas las cadenas estén completamente en minúsculas.

Es posible que queramos comprobar el número de duplicados verdaderos en la columna 'col_1' después de transformar su contenido a minúsculas::

import pandas as pd
df = pd.DataFrame({'col_1': ['A', 'B', 'A', 'A'], 'col_2': [1, 2, 2, 1]})

df['col_1'] = df['col_1'].str.lower()
print(df['col_1'].duplicated().sum())
2
Como se esperaba, obtuvimos 2 como salida. Esto se debe a que la tercera y la cuarta fila son idénticas a la primera.

Si quieres cambiar las mayúsculas y minúsculas en solo una parte de un string, puedes usar el método replace() que aprendiste en los sprints de Python básico. Por ejemplo, podrías cambiar todas las apariciones de 'GB' a 'gb' en la columna 'item' de los datos del teléfono:

import pandas as pd

stock = pd.read_csv('/datasets/phone_stock.csv')
stock['item'] = stock['item'].str.replace('GB', 'gb')

print(stock.head())
    id                  item             count
0  100480924     Apple iPhone Xr 64gb     10
1  100480924     Apple iPhone Xr 64gb     19
2  100480959     Xiaomi Redmi 6A 16gb     44
3  100480975          HUAWEI P30 lite     38
4  100480988  Samsung Galaxy A30 32gb     49

Si no estás seguro de cuál es el mejor enfoque, siempre puedes conservar la columna original y crear una nueva columna adicional, con las cadenas modificadas. Por ejemplo, podrías guardar el resultado de la sustitución en la columna 'item' en una nueva columna llamada 'item_modified':

import pandas as pd

stock = pd.read_csv('/datasets/phone_stock.csv')
stock['item_modified'] = stock['item'].str.replace('GB', 'gb')

print(stock.head())
    id                     item          count     item_modified
0  100480924     Apple iPhone Xr 64gb     10     Apple iPhone Xr 64gb
1  100480924     Apple iPhone Xr 64gb     19     Apple iPhone Xr 64gb
2  100480959     Xiaomi Redmi 6A 16gb     44     Xiaomi Redmi 6A 16gb
3  100480975          HUAWEI P30 lite     38          HUAWEI P30 lite
4  100480988  Samsung Galaxy A30 32gb     49  Samsung Galaxy A30 32gb

Utilicemos esta técnica para arreglar nuestro conjunto de datos de existencias de teléfonos.

Ejercicio 1

Siguiendo con el conjunto de datos phone_stock, haz lo siguiente:

Cambia los nombres de los modelos de teléfonos a minúsculas usando el método str.lower() y guárdalos en una nueva columna llamada 'item_lowercase', pero conserva también la columna original 'item'.

Imprime las primeras filas de la tabla actualizada y mira el resultado.

In [ ]:
import pandas as pd

df_stock = pd.read_csv('/datasets/phone_stock.csv')
df_stock['item_lowercase'] = df_stock['item'].str.lower()

print(df_stock.head())

"""Resultado
          id                     item  count           item_lowercase
0  100480924     Apple iPhone Xr 64gb     10     apple iphone xr 64gb
1  100480924     Apple iPhone Xr 64GB     19     apple iphone xr 64gb
2  100480959     Xiaomi Redmi 6A 16GB     44     xiaomi redmi 6a 16gb
3  100480975          HUAWEI P30 lite     38          huawei p30 lite
4  100480988  Samsung Galaxy A30 32GB     49  samsung galaxy a30 32gb"""

Ejercicio 2

Utilizando tu recién creada columna 'item_lowercase' y el método sum(), calcula el número total de dos modelos de teléfono:

1) 'apple iphone xr 64gb'

2) 'samsung galaxy a30 32gb'

Para contar el número de teléfonos Apple, filtra el DataFrame a partir de la columna 'item_lowercase' para incluir solo las filas con 'apple iphone xr 64gb' como valor. A continuación, extrae la columna 'count' del DataFrame filtrado y aplícale el método sum(). Almacena la cantidad total de teléfonos Apple en la variable apple.

Para los teléfonos Samsung, sigue el mismo procedimiento con la única diferencia de que guardes el número total de teléfonos Samsung en la variable samsung.

El precódigo ya contiene el código para imprimir tus resultados, no lo modifiques.

In [ ]:
import pandas as pd

df_stock = pd.read_csv('/datasets/phone_stock.csv')
df_stock['item_lowercase'] = df_stock['item'].str.lower()

apple = df_stock[df_stock['item_lowercase'] == 'apple iphone xr 64gb']["count"].sum()# completa esta línea de código
samsung = df_stock[df_stock['item_lowercase'] == 'samsung galaxy a30 32gb']["count"].sum()# completa esta línea de código

print("Número total de teléfonos Apple:", apple)
print("Número total de teléfonos Samsung:", samsung)

#Resultado
#Número total de teléfonos Apple: 29
#Número total de teléfonos Samsung: 60

Ahora elimina las filas con teléfonos duplicados llamando a drop_duplicates() en df_stock. Necesitamos eliminar filas únicamente en función de la columna item_lowercase', así que asegúrate de utilizarla como valor para el parámetro subset=.

Recuerda que después de eliminar los duplicados, tenemos que llamar al método reset_index() con el parámetro drop=True. Esto nos permite arreglar la indexación y eliminar el índice antiguo.

Por cierto, ¡puedes hacer todo esto con una sola línea de código! Puede ser un poco difícil, pero intenta encontrar la manera de hacerlo.

El resultado final debería asignarse de nuevo a df_stock. Imprime las primeras filas del df_stock cuando hayas terminado.

In [ ]:

import pandas as pd

df_stock = pd.read_csv('/datasets/phone_stock.csv')
df_stock['item_lowercase'] = df_stock['item'].str.lower()

df_stock = df_stock.drop_duplicates(subset='item_lowercase').reset_index(drop=True) # escribe tu código aquí

print(df_stock)# escribe tu código aquí)

"""Resultado
          id                     item  count           item_lowercase
0  100480924     Apple iPhone Xr 64gb     10     apple iphone xr 64gb
1  100480959     Xiaomi Redmi 6A 16GB     44     xiaomi redmi 6a 16gb
2  100480975          HUAWEI P30 lite     38          huawei p30 lite
3  100480988  Samsung Galaxy A30 32GB     49  samsung galaxy a30 32gb
4  100481020            Honor 8X 64GB     64            honor 8x 64gb"""

Ejercicio 4

¡Vamos a hacer algo realmente interesante! No te preocupes, estás preparado al 100% y nosotros te ayudaremos.

El precódigo incluye una línea que escribiste anteriormente para eliminar duplicados, por lo que ahora tenemos el DataFrame df_stock sin duplicados. Tu objetivo es establecer los valores correctos en la columna 'count' para las filas donde 'item' es 'Apple iPhone XR 64GB' y 'Samsung Galaxy A30 32GB'.

Los valores que vas a establecer ya fueron calculados por ti previamente y están almacenados en las variables apple y samsung en el precódigo.

La mejor forma de actualizar los valores de la columna 'count' es utilizar el atributo loc[] que puede reemplazar los valores en un lugar especificado.

Revisemos el df_stock después de eliminar los duplicados para ilustrar cómo loc[] puede ayudarnos a actualizar los valores:

Podemos utilizar loc[] para actualizar el valor de la primera fila (índice 0) y la columna 'count' del modelo Apple iPhone. Pasamos dos valores a loc[] para especificar el índice de la fila y el nombre de la columna, y luego utilizamos el signo = para establecer el valor deseado.

df_stock.loc[0,'count'] = 33
En el ejemplo anterior, utilizamos el valor 33, pero en realidad queremos establecer el valor de la variable apple que previamente hemos calculado y guardado.

Fue un ejemplo para el iPhone de Apple. El procedimiento para Samsung será el mismo, salvo que le pasaremos valores diferentes al atributo loc[].

In [ ]:
import pandas as pd

df_stock = pd.read_csv('/datasets/phone_stock.csv')
df_stock['item_lowercase'] = df_stock['item'].str.lower()

apple = df_stock[df_stock['item_lowercase'] == 'apple iphone xr 64gb']['count'].sum()
samsung = df_stock[df_stock['item_lowercase'] == 'samsung galaxy a30 32gb']['count'].sum()

df_stock = df_stock.drop_duplicates(subset='item_lowercase').reset_index(drop=True)

df_stock.loc[0, 'count']=29
df_stock.loc[3, 'count']=60

print(df_stock)# escribe tu código aquí

"""Resultado
          id                     item  count           item_lowercase
0  100480924     Apple iPhone Xr 64gb     29     apple iphone xr 64gb
1  100480959     Xiaomi Redmi 6A 16GB     44     xiaomi redmi 6a 16gb
2  100480975          HUAWEI P30 lite     38          huawei p30 lite
3  100480988  Samsung Galaxy A30 32GB     60  samsung galaxy a30 32gb
4  100481020            Honor 8X 64GB     64            honor 8x 64gb"""

Ejercicio 1

Steam es la plataforma de juegos para PC más popular del mundo, con una extensa colección de más de 6000 juegos y una vasta comunidad de millones de jugadores. Esta diversidad incluye desde grandes éxitos de la industria hasta títulos independientes más pequeños, lo que hace que contar con herramientas efectivas de descubrimiento sea un recurso extremadamente valioso. El conjunto de datos que utilizaremos contiene un registro de los comportamientos de los usuarios en Steam, incluyendo las siguientes columnas:

user-id: identificador único del usuario.

game-title: título del juego.

behavior-name: nombre del comportamiento (compra o jugar).

value: valor que indica la frecuencia o cantidad del comportamiento (1 para compras, otros valores para horas jugadas).

Tarea

Determina el número de filas duplicadas en el dataset. Elimina aquellas filas que se encuentren duplicadas.

Obtén el número de veces que cada juego aparece en la columna game-title. Utiliza value_counts() para ello.

Para reducir la variabilidad de los nombre de los juegos, utiliza el método str.lower() para estandarizar todos los registros de la columna game-title a minúsculas. Utiliza loc[] para actualizar la columna con los nuevos valores.

Pista**

Para acceder a una columna de un DataFrame, puedes usar la notación de corchetes, como df['nombre_columna']. Luego, utiliza .loc[] para acceder a todas las filas y a esa columna de manera directa, asegurándote de que los cambios se realicen sobre el DataFrame original. 
La notación sería df.loc[:, 'nombre_columna'].

In [ ]:
import pandas as pd

# Cargar los datos (asegúrate de reemplazar 'steam_videogames.csv' por el nombre correcto del archivo)
df = pd.read_csv('/datasets/steam-200k.csv')
df.columns = ['user_id', 'game_title', 'behavior_name', 'value']

# Mostrar DataFrame inicial
#print(df.head(5))

# 1. Eliminar filas duplicadas
print('Numero de filas duplicadas (Inicial): ', df.duplicated().sum())
df_sin_duplicados = df.drop_duplicates()

df_sin_duplicados.reset_index(drop=True, inplace=True)
print('Numero de filas duplicadas (Final): ', df_sin_duplicados.duplicated().sum())

# 2.Contar apariciones por título de juego
print("\n")
print(df_sin_duplicados['game_title'].value_counts())

# 3.Convertir títulos de juegos a minúsculas
df_sin_duplicados.loc[:,'game_title'] = df_sin_duplicados['game_title'].str.lower()

# Validación resultados
print("\n")
print(df_sin_duplicados.head(10))